# Introduction

<font color = green> <strong>MUST READ:</strong></font> This Notebook tutorial will allow you to add firmware validation capability to your application using TFLXTLS device.

Firmware validation support helps in avoiding the unauthorized firmware execution on a given system. The ability to securely upgrade the Microcontroller firmware starts with ensuring that the initial firmware has not been tampered with.

Secureboot command assists the MCU to identify fraudulent code installed on it. When this command support is implemented, the MCU can send either the digest or signature or both to the TFLXTLS. Then, the TFLXTLS validates this information and responds to host either with Success/Fail or with MAC value.

To have this functionality, once the firmware implementation is completed, it will be signed by the OEM signer to make the image authentic. On the product side, this application will be verified using OEM signers public key to ensure the authenticity. 

The notebook provides various steps involved to implement this using TFLXTLS device. Steps associated are,
1. Setup Modules and Hardware
2. Generating a Key pair (optional - If resources are already generated, this step is not required)
    * a. Device provisioning:load the firmware validation public key into the TFLXTLS reserved slot
3. Sign the firmware
4. Update the firmware to product
5. Verify the firmware

Before running this Notebook,
1. Make sure CryptoAuth Trust Platform is having factory reset program
2. Firmware validation project is complied and its hex is ready to use
3. Application project is moved to an offset to include Firmware validation project and its hex is ready to use

### Prerequisites:
This step of the tutorial will attempt to load all the necessary modules and their dependencies on your machine. If the modules are already installed you can safely step over the next Tutorial step.

<font color = orange> <strong>Note</strong></font> 
1. Installation time for prerequisites depends upon system and network speed.
2. Installing prerequisites for the first time takes more time and watch the kernel status for progress. Following image helps to locate the Kernel status,
<center><img src="../../../assets/notebook/img/kerner_status.png" alt="**Check Kernel Status**" /></center>


3. Installing prerequisites gives the following error and it can be safely ignored. Functionality remains unaffected.
    - <font color = orange> azure-cli 2.0.76 has requirement colorama~=0.4.1, but you'll have colorama 0.3.9 which is incompatible.</font>
    - <font color = orange>  azure-cli 2.0.76 has requirement pytz==2019.1, but you'll have pytz 2019.3 which is incompatible. </font>

In [ ]:
import sys, os

home_path = os.path.dirname(os.path.dirname(os.path.dirname(os.path.realpath(os.getcwd()))))
module_path = os.path.join(home_path, 'assets', 'python')
if not module_path in sys.path:
    sys.path.append(module_path)

from requirements_helper import requirements_installer
obj = requirements_installer(os.path.join(home_path, 'assets', 'requirements.txt'))
from trustplatform import program_flash, common_helper
programmer = program_flash()

### Setup Modules and Hardware
This step loads the required modules and helper functions to perform the resource generation sequence. It also 
1. Initializes the interface with TFLXTLS hardware and establishes commmunication with TFLXTLS.
2. Performs device initialization to verify the actual device attached

In [ ]:
import os, fnmatch, binascii
import ipywidgets as widgets
from ipywidgets import FileUpload
import cryptography
from cryptoauthlib import *
from common import *
from sboot_helper import *
from IPython.core.display import display
import hexrec.records as hr
import hexrec.blocks as hb
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import ec, utils
import warnings
warnings.filterwarnings('ignore')

# Setup cryptography 
crypto_be = cryptography.hazmat.backends.default_backend()
print("Importing modules - Successful")

####################################### Initialize interface
cfg = cfg_ateccx08a_kithid_default()
cfg.devtype = get_device_type_id('ATECC608A')
cfg.cfg.atcahid.dev_interface = ATCAKitType.ATCA_KIT_I2C_IFACE
cfg.cfg.atcahid.dev_identity = 0x6C #TFLXTLS device I2C address
assert_msg = "Check switch position on CryptoAuth Trust Platform board, verify connections"
if atcab_init(cfg) != ATCA_SUCCESS:
    # Check for CryptoAuth Trust Platform connection and default factory reset image
    status = programmer.check_for_factory_program()
    working_dir = os.getcwd()
    common_helper.delete_mplablog_files(working_dir)
    assert status == 'success', status
    assert atcab_init(cfg) == ATCA_SUCCESS, assert_msg

####################################### Verify attached device
# Get connected device type
info = bytearray(4)
assert_msg = "Cannot read the TFLXTLS device information, Verify device connections on Trust Platform"
assert atcab_info(info) == ATCA_SUCCESS, assert_msg

dev_type = get_device_type_id(get_device_name(info))
# Checking if the connected device matches with selected device
if dev_type != cfg.devtype:
    print('Device is not TFLXTLS, Connect TFLXTLS device')
    assert atcab_release() == ATCA_SUCCESS, 'atcab_release failed... Rerun Notebook'
   
print("Device initialization - Successful")

### Generating a firmware signing key pair

The most important step for firmware validation is having an ECC P256 key pair. The key pair can be generated using many of the options available like openssl (or) python modules etc..,. The private key will sign the image, the associated public key will verify the signed image.

<font color=red>This step is already perfomed part of resource generation using TFLXTLS_resource_generation\TFLXTLS Resource Generator.ipynb. </font> This key pair will be used for sign/verify operations in the steps to come. 

<font color = green>The keys are generated here using python module and is stored in computer's hard disk, this is not secure and is not recommended for actual production. In actual production environment these keys need to be stored in a secure storage as depitected below.</font>

<center><img src="img\firmware_validation_1.png" alt="**Get your Secure Elements here!**" /></center>

The following cell reads the firmware signing key pair generated and prints the associated public key

In [ ]:
key_file = '../../00_resource_generation/slot_15_ecc_key_pair.pem'
with open(key_file, 'rb') as f:
    # Load the public key from key file
    priv_key = serialization.load_pem_private_key(
            data=f.read(),
            password=None,
            backend=default_backend())
    public_key = priv_key.public_key().public_numbers().encode_point()[1:]

print("Public Key:")
print(pretty_print_hex(public_key, indent='    '))

### a. Device Provisioning: Load the firmware signing public key into TrustFLEX device

The public key generated in [section 2.2](#2.2.-Generating-a-firmware-signing-key-pair) needs to be loaded into TrustFLEX device. This public key will be used to verify the signed image in the steps to follow. 

The TrustFLEX device comes preconfigured. The secure boot public key is expected to be loaded into the slot #15. Please refer to TrustFLEX datasheet on www.microchip.com website for detailed slot organization. 

In addition, the device is configured for FullDigest secure boot operation, which means after first successful secure boot, digest is stored and on subsequent boots, the signature verification will be skipped and the device will perform only a digest comparison. Therefore, only the firmware validation public key will need to be uploaded into TrustFLEX. After this step, you will end up with a system consisting of the following cryptographic assets

<font color=red>This step is already perfomed part of resource generation using TFLXTLS_resource_generation\TFLXTLS Resource Generator.ipynb. </font>

<center><img src="img/firmware_validation_2.png" alt="**Get your Secure Elements here!**" /></center>

### Sign the firmware

Once the target firmware development is complete, firmware needs to be secured by signing its digest with the private key generated.

In this step the following things need to be done:
1. Get the firmware validation and application image
2. Combine the images and hash it with the SHA256 algorithm
3. Sign the digest

<font color=purple>slot_15_ecc_key_pair.pem file created in section [2.2](#2.2.-Generating-a-firmware-signing-key-pair) is required to run the following code.</font>

The following block diagram provides the high level flow of signing operation on the OEM side.

<center><img src="img/firmware_validation_3.png" alt="**Get your Secure Elements here!**" /></center>

### Load Hex files to Combine and Sign

In [ ]:
firmvalid_img_object = FileUpload(description='Step1a. Load Firmware Validation Hex', accept='.hex', layout=widgets.Layout(width='auto'), multiple=False)
app_img_object = FileUpload(description='Step1b. Load Application Hex', accept='.hex', layout=widgets.Layout(width='auto'), multiple=False)

def combine_hex(b):
    assert_msg = '''Its required both Firmware validation and 
    Application hex files are selected before running this''' 
    validity = any(firmvalid_img_object.value) & any(app_img_object.value)
    print('Upload Firmware validation and Application Hex file')
    display(widgets.Valid(value=validity, description='Upload'))
    assert validity, assert_msg
    combine_sign_hex(firmvalid_img_object, app_img_object)

def flash_combine_hex(b):
    if not os.path.isfile("combined_image.hex"):
        print("Combined hex file needs to be generated before running this step")
        flash_hex.button_style = 'danger'
        return None
    # Flash combine hex into CryptoAuth Trust Platform
    if programmer.check_mplab_path() == True:
        print("Programming combined hex into CryptoAuth Trust Platform board")
        mergehex_path = os.path.join(os.path.realpath(os.getcwd()), "combined_image.hex")
        processout = programmer.flash_micro(mergehex_path)
        working_dir = os.getcwd()
        common_helper.delete_mplablog_files(working_dir)
        print(processout.stdout)
        if processout.returncode != 0:
            flash_hex.button_style = 'danger'
            assert processout.returncode == 0, processout.stderr
        flash_hex.button_style = 'success'
        print("Programmed hex successfully")
    else:
        flash_hex.button_style = 'danger'
        print("Program the CryptoAuth Trust Platform board with combined hex")       

tooltip = '''Combines both hex files and Signs the combined image,
Should run only after loading both hex file'''
combine_and_sign = widgets.Button(description = "Step1c. Combine both HEX and Sign", tooltip=tooltip, layout=widgets.Layout(width='auto'))
combine_and_sign.on_click(combine_hex)

tooltip = "Flash the combined hex into CryptoAuth Trust Platform"
flash_hex = widgets.Button(description = "Flash combine hex", tooltip=tooltip, layout=widgets.Layout(width='auto'))
flash_hex.on_click(flash_combine_hex)
display(widgets.VBox((firmvalid_img_object, app_img_object, combine_and_sign)))

### Update the firmware to product
This step upgrades the new firmware image digest to Product secure element. The digest and signature generated in the previous step will be provided to secure element using Secure boot command.  with option set to store (Full Copy) on successful validation of the digest and signature.

#### Bus obfuscation
To ensure the digest is protected on the bus, its possible to obfuscate it while sending on the bus. <strong>IO Protection Key</strong> helps in encrypting the data.

On the TrustFLEX device Slot6 is configured to hold the IO protection key. This slot is configured to Never read and option to lock after writing the key. This is a unique key on the given setup. Every host and device pair agrees on a common key during first power up. Once the pair agrees on, this slot will be locked to avoid further updates. This ensures only this host and device pair knows the key. 

Any compromise of this key by the host, limits the impact to only this pair. All other host and device security remains intact.

<center><img src="img/firmware_validation_5.png" alt="**Get your Secure Elements here!**" /></center>

<strong>Note:</strong> A click button is provided at the end of the notebook to upgrade the image. This is done to easily access the upgrade and verify operations together.

### Verify the application image

This step checks the application to execute is valid or not by issuing Secure boot command to secure element.

<center><img src="img/firmware_validation_4.png" alt="**Get your Secure Elements here!**" /></center>

Below code feeds required parameters for secureboot operation on TrustFLEX device. On receiving this, TrustFLEX performs digest store or comparison based on the mode.

In [ ]:
tooltip='Click to update digest/signature using Secureboot command'
firmware_update = widgets.Button(description='Firmware Update',tooltip=tooltip)
tooltip= 'Click to Verify using Secureboot command'
firmware_verify = widgets.Button(description='Firmware Verify',tooltip=tooltip)

io_prot_key = get_symmetric_key('../../00_resource_generation/slot_6_secret_key.pem')

def secureboot_update(b):
# Generating a random number to use
    if not os.path.exists("combined_image.hex"):
        print("The application binary and firmware validation binary needs to be combined and signed before this step")
        firmware_update.button_style = 'danger'
        return None
    digest, signature = get_digest_signature()
    host_random = os.urandom(32)
    is_verified = AtcaReference(False)
    # In real application, this step should be carried ONLY on firmware upgrade
    print('Perform Application upgrade request... ')
    atcab_secureboot_mac(SECUREBOOT_MODE_FULL_COPY, digest, signature, host_random, io_prot_key, is_verified)
    if 1 == bool(is_verified.value):
        print('Secureboot Update Success...')
        firmware_update.button_style = 'success'
    else:
        firmware_update.button_style = 'danger'
        print('Secureboot Update failed...')

def secureboot_verify(b):
    # Generating a random number to use
    if not os.path.exists("combined_image.hex"):
        print("The application binary and firmware validation binary needs to be combined and signed before this step")
        firmware_update.button_style = 'danger'
        return None
    digest, signature = get_digest_signature()
    host_random = os.urandom(32)
    is_verified = AtcaReference(False)
    # Perform Secureboot operation on the application file
    print('Perform Application validation request... ')
    assert atcab_secureboot_mac(SECUREBOOT_MODE_FULL_STORE, digest, signature, host_random, io_prot_key, is_verified) == ATCA_SUCCESS
    if 1 == bool(is_verified.value):
        print('Secureboot Verify Success...')
        firmware_verify.button_style = 'success'
    else:
        firmware_verify.button_style = 'danger'
        print('Secureboot Verify failed...')

firmware_update.on_click(secureboot_update)
firmware_verify.on_click(secureboot_verify)
display(widgets.HBox((firmware_update, firmware_verify, flash_hex)))